![clothing_classification](clothing_classification.png)


Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.

In [1]:
# Run the cells below first

In [2]:
!pip install torchmetrics
!pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.4/891.4 kB 57.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [4]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

def to_rgb(image):
    return image.convert("RGB")

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.Compose([
    transforms.Lambda(to_rgb),
    #transforms.RandomVerticalFlip(p=0.5),
    #transforms.RandomRotation(90),
    #transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])
                                  )
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.Compose([
    transforms.Lambda(to_rgb),
    transforms.Resize((64,64)),
    transforms.ToTensor()
])
                                 )

In [5]:
#model architecture
class MultiClassImageClassifier(nn.Module):
    def __init__(self, num_classes):
        super(MultiClassImageClassifier, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=4, stride=4),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten()  
        )
        self.classifier = nn.Sequential(
            nn.Linear(32*8*8, num_classes),
            nn.Softmax(dim=-1)
        )
    def forward(self,x):
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x
            

In [6]:
# Get the number of classes

classes = train_data.classes 
print(classes) 
num_classes = len(train_data.classes) 
print(num_classes)

['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
10


In [7]:

dataloader_train = DataLoader(dataset=train_data, batch_size=16, shuffle=True)
num_classes = 10
#defind the model
model = MultiClassImageClassifier(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epoch_loss = []
running_loss = 0.0
model.train()
for epoch in range(2):
    for images, labels in dataloader_train:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_loss.append(running_loss / len(dataloader_train))
    running_loss = 0.0

print(epoch_loss)

[1.6446020466486613, 1.600796412118276]


In [8]:

dataloader_test = DataLoader(dataset=test_data, batch_size=16, shuffle=True)

metric_accuracy = Accuracy(task="multiclass", num_classes=num_classes)
metric_precision = Precision(task="multiclass", num_classes=num_classes, average=None)
metric_recall = Recall(task="multiclass", num_classes=num_classes, average=None)

predictions = []

model.eval()
with torch.no_grad():
    for images, labels in dataloader_test:
        outputs = model.forward(images)
        preds = torch.argmax(outputs, dim=-1)
        predictions.extend(preds.tolist())
        metric_accuracy(preds, labels)
        metric_precision(preds, labels)
        metric_recall(preds, labels)
        
accuracy = metric_accuracy.compute().item()
print(f"accuracy: {accuracy}")

accuracy: 0.8622999787330627


In [9]:
precision = metric_precision.compute()
recall = metric_recall.compute()

precision_per_class = {
    k: precision[v].item()
    for k, v 
    in test_data.class_to_idx.items()
}
print(f"precison per class:\n{precision_per_class}\n")


recall_per_class = {
    k: recall[v].item()
    for k, v 
    in test_data.class_to_idx.items()
}
print(f"recall pers class:\n{recall_per_class}")

precison per class:
{'T-shirt/top': 0.8178178071975708, 'Trouser': 0.9795709848403931, 'Pullover': 0.7798253893852234, 'Dress': 0.7943686246871948, 'Coat': 0.813238799571991, 'Sandal': 0.9813277721405029, 'Shirt': 0.6270492076873779, 'Sneaker': 0.9297561049461365, 'Bag': 0.9625884890556335, 'Ankle boot': 0.9430814385414124}

recall pers class:
{'T-shirt/top': 0.8169999718666077, 'Trouser': 0.9589999914169312, 'Pullover': 0.8040000200271606, 'Dress': 0.9309999942779541, 'Coat': 0.6880000233650208, 'Sandal': 0.9459999799728394, 'Shirt': 0.6119999885559082, 'Sneaker': 0.953000009059906, 'Bag': 0.9520000219345093, 'Ankle boot': 0.9610000252723694}
